# Evaluating a model

In this example notebook we'll evaluate the model with the most popular metrics.

## Downloading poquad dataset

In [38]:
from datasets import load_dataset

poquad = load_dataset("clarin-pl/poquad")

In [39]:
poquad_validation = poquad["validation"]

print("Context: ", poquad_validation[0]["context"])
print("Question: ", poquad_validation[0]["question"])
print("Answer: ", poquad_validation[0]["answers"])

Context:  Pisma rabiniczne – w tym Miszna – stanowią kompilację poglądów różnych rabinów na określony temat. Zgodnie z wierzeniami judaizmu Mojżesz otrzymał od Boga całą Torę, ale w dwóch częściach: jedną część w formie pisanej, a drugą część w formie ustnej. Miszna – jako Tora ustna – była traktowana nie tylko jako uzupełnienie Tory spisanej, ale również jako jej interpretacja i wyjaśnienie w konkretnych sytuacjach życiowych. Tym samym Miszna stanowiąca kodeks Prawa religijnego zaczęła równocześnie służyć za jego ustnie przekazywany podręcznik.
Question:  Czym są pisma rabiniczne?
Answer:  {'text': ['kompilację poglądów różnych rabinów na określony temat'], 'answer_start': [43]}


In [42]:
poquad_validation.filter(lambda x: len(x["answers"]["text"]) != 1)

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 0
})

Dla każdego pytania z zestawu walidacyjnego istnieje tylko jedna odpowiedź. Jednak dla niektórych zestaów pytań poprawnym może być kilka odpowiedzi.

In [43]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model_path = './output/roberta-base-squad2-pl/checkpoint-8500'

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForQuestionAnswering.from_pretrained(model_path)

In [44]:
print(tokenizer.is_fast)

True


In [45]:
context = poquad_validation[0]["context"]
question = poquad_validation[0]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

'<s>Czym są pisma rabiniczne?</s></s>Pisma rabiniczne – w tym Miszna – stanowią kompilację poglądów różnych rabinów na określony temat. Zgodnie z wierzeniami judaizmu Mojżesz otrzymał od Boga całą Torę, ale w dwóch częściach: jedną część w formie pisanej, a drugą część w formie ustnej. Miszna – jako Tora ustna – była traktowana nie tylko jako uzupełnienie Tory spisanej, ale również jako jej interpretacja i wyjaśnienie w konkretnych sytuacjach życiowych. Tym samym Miszna stanowiąca kodeks Prawa religijnego zaczęła równocześnie służyć za jego ustnie przekazywany podręcznik.</s>'

In [87]:
import evaluate

metric = evaluate.load("squad")

In [92]:
from transformers import pipeline

def get_predictions(eval_dataset):
    qa_pipeline = pipeline('question-answering', model=model, tokenizer=tokenizer)

    predictions = []
    for question, context, id in zip(eval_dataset["question"], eval_dataset["context"], eval_dataset["id"]):
        answer = qa_pipeline(question=question, context=context)
        prediction = {
            'id': id,
            'prediction_text': answer['answer']
        }

        predictions.append(prediction)

    return predictions

In [106]:
part_datset = poquad_validation.select(range(1000))

predictions = get_predictions(part_datset)

In [107]:
theoretical_answers = [
    {"id": ex["id"], "answers": ex["answers"]} for ex in part_datset
]

In [108]:
metric.compute(predictions=predictions, references=theoretical_answers)

{'exact_match': 34.6, 'f1': 51.20063731342978}